In [ ]:
# %env SPARK_HOME=C:\MapProdQA\spark-3.5.0-bin-hadoop3
# %env SPARK_HOME=C:\Python311\Lib\site-packages\pyspark
# %env HADOOP_HOME=C:\MapProdQA\spark-3.5.0-bin-hadoop3
# %env HADOOP_HOME=C:\Users\anilk\Downloads\hadoop-2.8.1
# %env JAVA_HOME="C:\Program Files\Amazon Corretto\jdk1.8.0_282"
# %env PATH=%PATH%;%SPARK_HOME%\bin
# %env PYTHONPATH=%PYTHONPATH%;"%SPARK_HOME%\python"
# %env PYTHONPATH=%PYTHONPATH%;"%SPARK_HOME%\python\lib\*.zip"

# optional
# %env PYSPARK_PYTHON=python
%env PYSPARK_DRIVER_PYTHON=C:\Python311\python.exe
# %env PYSPARK_DRIVER_PYTHON=jupyter 
# %env PYSPARK_DRIVER_PYTHON_OPTS=notebook
# %env SPARK_REMOTE=""
# PYTHONPATH=$SPARK_HOME$\python;$SPARK_HOME$\python\lib\py4j-<version>-src.zip
# %env PYSPARK_SUBMIT_ARGS=--master local[1] pyspark-shell
# %env PYSPARK_SUBMIT_ARGS=C:\MapProdQA\Python3.9\Lib\site-packages\pyspark\jars\*.jar
# %env SPARK_CLASSPATH=C:\MapProdQA\Python3.9\Lib\site-packages\pyspark\jars\*.jar

env: PYSPARK_DRIVER_PYTHON=C:\Python311\python.exe


In [1]:
!echo %SPARK_HOME% 
!echo %PYSPARK_PYTHON% 
!echo %HADOOP_HOME% 
!echo %JAVA_HOME% 
!echo %PYTHONPATH%
!echo %PYSPARK_DRIVER_PYTHON%
!echo %PYSPARK_SUBMIT_ARGS%

C:\Spark\spark-3.5.1-bin-hadoop3
%PYSPARK_PYTHON%
C:\Hadoop\hadoop-3.3.6\
C:\Program Files\Java\jdk-17
C:\Users\lucio\AppData\Local\Programs\Python\Python312;C:\Users\lucio\AppData\Local\Programs\Python\Python312
%PYSPARK_DRIVER_PYTHON%
%PYSPARK_SUBMIT_ARGS%


In [2]:
import findspark
import os
import pandas as pd
# set the environment variables
findspark.init()

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F 
from IPython.display import display

spark = SparkSession.builder.master("local[*]").appName('PySparkShell').getOrCreate()
# set master to local{*} to tell that local machine is the master node

# https://stackoverflow.com/questions/46125604/databricks-display-function-equivalent-or-alternative-to-jupyter
# While practise, to see the dataframe without .show(). This will enable the display()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

spark

In [6]:
# to read csv from spark
path = "data.csv"
output_path = "output_data.csv"
import sys

df = spark.read.csv(path, inferSchema=True, header=True)

# to save a csv
df.write.format('csv').mode("overwrite").option("header",True).save(os.path.join(os.getcwd(), 'new_data.csv'))
display(df)

Name,Geometry
Place1,MULTIPOINT ((19.5...
Place2,LINESTRING (10 10...
Place3,MULTIPOINT ((19.5...
Place4,MULTIPOLYGON (((0...
Place5,LINESTRING (10 10...
Place6,POLYGON ((19.5112...
Place7,POLYGON ((19.5112...


In [2]:
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.spark import *
path = 'data.csv'
spark = SparkSession. \
    builder. \
    appName('appName'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.5.0,'
           'org.datasyslab:geotools-wrapper:1.5.0-28.2'). \
    getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
SedonaContext.create(spark)
df = spark.read.csv(path, inferSchema=True, header=True)
df.createOrReplaceTempView("realdf")
df1 = spark.sql(
    '''SELECT ST_GeometryType(ST_GeomFromText(geometry)) as geometry_type,
    COUNT(SUBSTRING_INDEX(geometry, ' ', 1)) AS geometry_instance_count 
    FROM realdf 
    WHERE geometry is NOT NULL
    GROUP BY geometry_type''')
display(df1)

geometry_type,geometry_instance_count
ST_MultiPolygon,1
ST_LineString,2
ST_MultiPoint,2
ST_Polygon,2


In [3]:
df1.show()

+---------------+-----------------------+
|  geometry_type|geometry_instance_count|
+---------------+-----------------------+
|ST_MultiPolygon|                      1|
|  ST_LineString|                      2|
|  ST_MultiPoint|                      2|
|     ST_Polygon|                      2|
+---------------+-----------------------+



In [4]:
spark.stop()
spark.sparkContext.stop()